<a href="https://colab.research.google.com/github/learn2Pro/rl_learning/blob/master/model_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 参考
  - https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html
## outline
- 数据并行是切数据（scattering inputs and gathering outputs），模型并行是切模型（shards）；
  - 模型并行：单卡放不下一份模型；
  - 将一份大模型，不同的层s切分到不同的卡上；
- device_map：Huggingface
- 模型并行 on ToyModel
- 模型并行：on ResNet
  - 不需要引入额外的 torch api 支持；
## huggingface 的支持
### device_map
- "auto", "balanced", "balanced_low_0", "sequential"
- auto
  - GPU(s) > CPU (RAM) > Disk.

In [1]:
#!pip install transformers
#!pip install accelerate
#!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
for i, para in enumerate(model.named_parameters()):
#     print(f'{i}, {para[0]}\t {para[1].device} \t{para[1].dtype}')
    print(f'{i}, \t {para[1].device} \t{para[1].dtype}')

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

0, 	 cuda:0 	torch.float16
1, 	 cuda:0 	torch.int8
2, 	 cuda:0 	torch.int8
3, 	 cuda:0 	torch.int8
4, 	 cuda:0 	torch.int8
5, 	 cuda:0 	torch.int8
6, 	 cuda:0 	torch.int8
7, 	 cuda:0 	torch.int8
8, 	 cuda:0 	torch.float16
9, 	 cuda:0 	torch.float16
10, 	 cuda:0 	torch.int8
11, 	 cuda:0 	torch.int8
12, 	 cuda:0 	torch.int8
13, 	 cuda:0 	torch.int8
14, 	 cuda:0 	torch.int8
15, 	 cuda:0 	torch.int8
16, 	 cuda:0 	torch.int8
17, 	 cuda:0 	torch.float16
18, 	 cuda:0 	torch.float16
19, 	 cuda:0 	torch.int8
20, 	 cuda:0 	torch.int8
21, 	 cuda:0 	torch.int8
22, 	 cuda:0 	torch.int8
23, 	 cuda:0 	torch.int8
24, 	 cuda:0 	torch.int8
25, 	 cuda:0 	torch.int8
26, 	 cuda:0 	torch.float16
27, 	 cuda:0 	torch.float16
28, 	 cuda:0 	torch.int8
29, 	 cuda:0 	torch.int8
30, 	 cuda:0 	torch.int8
31, 	 cuda:0 	torch.int8
32, 	 cuda:0 	torch.int8
33, 	 cuda:0 	torch.int8
34, 	 cuda:0 	torch.int8
35, 	 cuda:0 	torch.float16
36, 	 cuda:0 	torch.float16
37, 	 cuda:0 	torch.int8
38, 	 cuda:0 	torch.int8
39, 	 cu

## toy model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
class ToyModel(nn.Module):
  def __init__(self):
    super(ToyModel, self).__init__()
    self.net1 = nn.Linear(10000, 10).to(device)
    self.relu = F.relu
    self.net2 = nn.Linear(10, 5).to(device)
  def forward(self, x):
    x = self.relu(self.net1(x.to(device)))
    return self.net2(x.to(device))

In [17]:
model = ToyModel()
model

ToyModel(
  (net1): Linear(in_features=10000, out_features=10, bias=True)
  (net2): Linear(in_features=10, out_features=5, bias=True)
)

In [18]:
print(next(model.net1.parameters()).device)
print(next(model.net2.parameters()).device)

cuda:0
cuda:0


In [20]:
model = ToyModel()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 1e-3)

optimizer.zero_grad()
outputs = model(torch.randn((20,10000)))
labels = torch.randn((20,5)).to(device)
loss_fn(outputs, labels).backward()
optimizer.step()

## 4 split resnet
- summary of resnet
  - model = ResNet(block, layers, **kwargs)
  - # resnet18
  _resnet(BasicBlock, (2, 2, 2, 2))
  - # resnet34
  _resnet(BasicBlock, (3, 4, 6, 3))

  - # resnet50
  _resnet(Bottleneck, (2, 2, 2, 2))
  - # resnet101
  _resnet(Bottleneck, (2, 2, 2, 2))
  - # resnet152
  _resnet(Bottleneck, (2, 2, 2, 2))
  

In [13]:
import torch
from torch import nn
from torchvision.models.resnet import ResNet, Bottleneck

In [29]:
# resnet50
model = ResNet(Bottleneck, [3, 4, 6, 3])
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
from torchsummary import summary
summary(model, input_size=(3, 128, 128), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

In [25]:
t = torch.randn((2, 3, 4))
t.view(t.shape[0],-1).shape

torch.Size([2, 12])

In [45]:
class ModelParallelResNet50(ResNet):
    def __init__(self, num_classes=1000):
        super().__init__(Bottleneck, [3, 4, 6, 3], num_classes=num_classes)
        # conv1 => bn1 => relu => maxpool => layer1-layer4 => avgpool => fc
        self.seq1 = nn.Sequential(
            self.conv1,
            self.bn1,
            self.relu,
            self.maxpool,
            self.layer1,
            self.layer2
        ).to('cuda:0')

        self.seq2 = nn.Sequential(
            self.layer3,
            self.layer4,
            self.avgpool,
        ).to('cuda:1')

        self.fc.to('cuda:1')

    def forward(self, x):
        # model parts（layers） 的一个（卡间）串行，
        x = self.seq2(self.seq1(x).to('cuda:1'))
        return self.fc(x.view(x.size(0), -1))

In [46]:
def model_size(model):
  return sum([para.numel() for para in model.parameters()])

In [47]:
model_size(model)

25557032

### 4.2 train pipeline

In [48]:
num_classes = 10
one_hot_indices = torch.LongTensor(5) \
                           .random_(0, num_classes) \
                           .view(5, 1)
one_hot_indices

tensor([[9],
        [8],
        [3],
        [8],
        [4]])

In [49]:
labels = torch.zeros(5, num_classes) \
                      .scatter_(1, one_hot_indices, 1)
labels

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [50]:
num_classes = 1000
num_batches = 3
batch_size = 120
image_w = 128
image_h = 128


def train(model):
    model.train(True)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    one_hot_indices = torch.LongTensor(batch_size) \
                           .random_(0, num_classes) \
                           .view(batch_size, 1)

    for _ in range(num_batches):
        # generate random inputs and labels
        # (b, c, w, h)
        inputs = torch.randn(batch_size, 3, image_w, image_h)
        # one hot：(batch_size, num_classes)，行粒度只有一个为1 （one-hot）
        # scatter 的三个参数：dim, index, src
        labels = torch.zeros(batch_size, num_classes) \
                      .scatter_(1, one_hot_indices, 1)

        # run forward pass
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
#         print('outputs', outputs.shape)
        # run backward pass
        labels = labels.to(outputs.device)
        loss_fn(outputs, labels).backward()
        optimizer.step()

In [51]:
import matplotlib.pyplot as plt
# plt.switch_backend('Agg')
import numpy as np
import timeit

num_repeat = 10

stmt = "train(model)"

# 模型并行
setup = "model = ModelParallelResNet50()"
mp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
mp_mean, mp_std = np.mean(mp_run_times), np.std(mp_run_times)

# 单卡
setup = "import torchvision.models as models;" + \
        "model = models.resnet50(num_classes=num_classes).to('cuda:0')"
rn_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
rn_mean, rn_std = np.mean(rn_run_times), np.std(rn_run_times)


def plot(means, stds, labels, fig_name):
    fig, ax = plt.subplots()
    ax.bar(np.arange(len(means)), means, yerr=stds,
           align='center', alpha=0.5, ecolor='red', capsize=10, width=0.6)
    ax.set_ylabel('ResNet50 Execution Time (Second)')
    ax.set_xticks(np.arange(len(means)))
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    plt.tight_layout()
#     plt.savefig(fig_name)
#     plt.close(fig)

RuntimeError: ignored